In [ ]:
import pandas as pd
import json
import cv2
import re
import pylab as pl
import tqdm
import pickle
from collections import defaultdict

In [ ]:
preds = pd.read_csv('data/predictions/nonresidential-1.tsv', sep='\t')
preds = preds[preds.num_boxes>0]
preds['max_prob'] = preds.probs.apply(lambda r: max(json.loads(r)))
preds

In [ ]:
# Get GPS -> image_ids list
#with open('gsv/metadata.pkl', 'rb') as f:
#    metadata = pickle.load(f)
#    
#gps_to_ids = defaultdict(list)
#for image_id,obj in metadata.items():
#    gps = tuple(map(float,obj['location'].split(',')))
#    gps_to_ids[gps].append(image_id)

In [ ]:
for i,row in enumerate(preds.sort_values('probs').iloc[::-1].itertuples()):
    if i<180: continue

    filename = row.File
    image_id = int(re.search('image0*(\d+).jpg', filename).groups()[0])
    
    print(i)
    print(filename)
    
    # Display location
    image_num = int(re.search('image0*(\d+)\.jpg', filename).groups()[0])
    #gps = metadata[image_num]['location']
    gps = (row.Latitude, row.Longitude)
    print(str(gps)[1:-1])

    #sample = id_to_obj[image_id]
    #path = sample['file_name'] 
    path = f'gsv/{filename}'
    img = cv2.imread(path)
    
    ################
    # Show Image
    pl.figure(figsize=(34,18))
    pl.imshow(img[:, :, ::-1])
        
    ################
    # Predicted Boxes
    probs   = json.loads(row.probs)
    boxes   = json.loads(row.boxes)
    classes = json.loads(row.classes)

    #colors = ['r', 'deepskyblue', 'b', 'g', 'c', 'm', 'y', 'tab:orange', 'tab:pink', 'goldenrod']
    colors = ['red', 'yellow']
    counter = 0
    for j,box in sorted(enumerate(boxes), key=lambda t:probs[t[0]], reverse=True):
        # Only show the 7 highest-probability boxes
        counter += 1
        if counter > 7: break
            
        #color = 'r' # colors[j]
        pred_label = classes[j]
        color = colors[pred_label]
        prob = probs[j]

        # Plot rectangle
        x1,y1,x2,y2 = box
        rectangle = pl.Rectangle((x1,y1), x2-x1, y2-y1, ec=color, facecolor='None', lw=1+10*prob)
        pl.gca().add_patch(rectangle)

        # Plot probability
        pl.text(x1, y1*.99, f'{int(prob*100)}%', c=color, fontsize=6+50*prob)
            
    pl.show()
    
    probs = json.loads(row.probs)
    score = max(probs)
    if score <0.6: break

In [ ]:
preds[abs(preds.Latitude-42.335479)<.00001]

In [ ]:
for i,row in enumerate(preds.sort_values('probs').iloc[::-1].itertuples()):
    if i<180: continue

    filename = row.File
    image_id = int(re.search('image0*(\d+).jpg', filename).groups()[0])
    
    print(i)
    print(filename)
    
    # Display location
    image_num = int(re.search('image0*(\d+)\.jpg', filename).groups()[0])
    #gps = metadata[image_num]['location']
    gps = (row.Latitude, row.Longitude)
    print(str(gps)[1:-1])

    #sample = id_to_obj[image_id]
    #path = sample['file_name'] 
    path = f'gsv/{filename}'
    img = cv2.imread(path)
    
    ################
    # Show Image
    pl.figure(figsize=(34,18))
    pl.imshow(img[:, :, ::-1])
        
    ################
    # Predicted Boxes
    probs   = json.loads(row.probs)
    boxes   = json.loads(row.boxes)
    classes = json.loads(row.classes)

    #colors = ['r', 'deepskyblue', 'b', 'g', 'c', 'm', 'y', 'tab:orange', 'tab:pink', 'goldenrod']
    colors = ['red', 'yellow']
    counter = 0
    for j,box in sorted(enumerate(boxes), key=lambda t:probs[t[0]], reverse=True):
        # Only show the 7 highest-probability boxes
        counter += 1
        if counter > 7: break
            
        #color = 'r' # colors[j]
        pred_label = classes[j]
        color = colors[pred_label]
        prob = probs[j]

        # Plot rectangle
        x1,y1,x2,y2 = box
        rectangle = pl.Rectangle((x1,y1), x2-x1, y2-y1, ec=color, facecolor='None', lw=1+10*prob)
        pl.gca().add_patch(rectangle)

        # Plot probability
        pl.text(x1, y1*.99, f'{int(prob*100)}%', c=color, fontsize=6+50*prob)
            
    pl.show()
    
    probs = json.loads(row.probs)
    score = max(probs)
    if score <0.6: break

# Load Manually-checked Results of Predictions

In [ ]:
confirmed_df = pd.read_csv('data/pred_cameras_wboag.tsv', sep='\t')
confirmed_df['max_prob'] = confirmed_df['probs'].apply(lambda t: float(t[1:-1].split(',')[0]))
confirmed_df['confirm dome'] = confirmed_df['confirm dome'].fillna(0)
confirmed_df['confirm button'] = confirmed_df['confirm button'].fillna(0)
confirmed_df['confirm box'] = confirmed_df['confirm box'].fillna(0)
#confirmed_df = confirmed_df[(confirmed_df['confirm dome'] + confirmed_df['confirm button'] + confirmed_df['confirm box'])>0]
confirmed_df

In [ ]:
ranges = [[0.9,1.01], [0.8,0.9], [0.7,0.8], [0.7,1.0]]
for low,high in ranges:
    rows = confirmed_df[(confirmed_df.max_prob>=low)&(confirmed_df.max_prob<high)]
    crows = rows[(rows['confirm dome'] + rows['confirm button'] + rows['confirm box'])>0]

    frac = len(crows) / len(rows)
    print(f'[{low},{high}) n={len(rows)}, {frac*100:.1f}% of images had cameras')
    for col in ['confirm dome', 'confirm button', 'confirm box']:
        print(f'\t{col:14s}: {int(rows[col].sum()):3d}')
    print()